# Análise de correspondência simples e múltipla

## Setup de ambiente

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from scipy.stats import chi2_contingency
from statsmodels import api as sm
import prince

In [ ]:
DATAFOLDER_PATH: Path = Path('/app/data/unsupervised-learning/analise-correspondencia-simples-multipla')
DATAFILE: Path = DATAFOLDER_PATH / 'perfil_aplicacao.xlsx'

In [ ]:
df: pd.DataFrame = pd.read_excel(DATAFILE)

display(df['Perfil'].value_counts())
display(df['Tipo de Aplicação'].value_counts())
df.head()

## Análise de Correspondência

### Introdução



* Técnina adequada para a análise de variáveis categoricas (qualitativas);
  * Variáveis geradas por escala [Likert](https://en.wikipedia.org/wiki/Likert_scale);
    * Evita problema de poderação arbitrária;
    * Cada variável em escala Likert corresponde a uma variável na análise simples ou múltipla;
    * Exemplos:
      * Concordo plenamente, parcialmente, não concordo nem discord, discordo parcialmente, totalmente;
  * Exemplos de aplicação:
    * Faixas de renda;
    * Nível de escolaridade;
    * Tipo de cultura implmentada em solo;
    * Gravidade de sintomos de uma doença e comorbidades;
* Técnica exploratória, ou seja, não supervisionada;
  * Usada para avaliar a relação conjunta entre variáveis (interdependência)
  * Não são aplicáveis para inferência
    * Não há modelos do tipo $y_{i} = x_{1i} + x_{2i} + ... + u_{i}$;
* Se novas variáveis forem adicionadas, deve-se refazer a análise;

### Análise de Correspondência Simples (Anacor)

#### Teste $\chi^2$ para associaçõa

##### Tabela de contigência

* Contem as frequências absolutas observadas para cada par de categorias
  * Também referenciada como tabela de classificação cruzada (_cross table_)

In [ ]:
tabela: pd.DataFrame = pd.crosstab(df['Perfil'],
                                   df['Tipo de Aplicação'])

tabela.head()

In [ ]:
contingency_table: sm.stats.Table = sm.stats.Table(tabela)

contingency_table

In [ ]:
ca = prince.CA().fit(tabela)

ca

##### Tabela de frequências absolutas esperadas

* Para cada célula na [tabela de contigência](#tabela-de-contigencia), a frequência será dada pela formula:

$$
\text{Frequência absoluta esperada}_{ij}=\frac{\sum_{i=0}^{i+1}\sum_{j=0}^{j+1}}{N}
$$

> A multiplicação dos somatórios da linha e da coluna dividios pela quantidade total de registros

In [ ]:
contingency_table.fittedvalues

##### Tabela de resíduos

* Para cada célula referente a categoria $ij$ das duas variáveis, o valor do resíduo é dado pela formula:

$$
\text{Resíduo}_{ij}=n_{ij} - \frac{\sum_{i=0}^{i+1}\sum_{j=0}^{j+1}}{N}
$$

In [ ]:
tabela - contingency_table.fittedvalues

##### Tabela com os valores $\chi^2$

* Para cada valor de categoria $ij$, o valor da estatística $\chi^2$ é dado pela formula:

$$
\chi^2_{ij} = \frac{\text{Resíduos}_{ij}^2}{\text{Frequência absoluta esperada}_{ij}}
$$

* Posteriormente, os valores são somados para se obter o valor total de $\chi^2$;
  * O objetivo é verificar se existe relação estatisticamente significante;
    * $H_0$: As variáveis se associam de forma aleatória
    * $H_1$: As variáveis não se associam de forma aleatória
    * O valor para os graus de liberdade são dados por:
      * $\text{Graus de liberdade}=(i - 1) (j - 1)$

In [ ]:
chi_2, p_value, degrees_freedom, *_ = chi2_contingency(tabela)

labels: tuple[str] = ('chi square', 'p value', 'degrees of freedom')

for stats, label in zip((chi_2, p_value, degrees_freedom), labels):
  print(f'{label}: {stats:,.2f}')

In [ ]:
contingency_table.chi2_contribs

##### Tabela de resíduos pradronizados

* Para cada valor de categoria $ij$, p valor de resíduo padronizado é dado pela formula:

$$
\text{Resíduo padronizado}_{ij} = \frac{\text{Resíduo}_{ij}}{\sqrt{\text{Frequência absoluta esperada}_{ij}}}
$$

In [ ]:
contingency_table.resid_pearson

##### Tabela de resíduos pradronizados ajustados

* A análise de resíduos padronizados ajustados permite aprofundar a análise com foco nas categorias das variáveis;
  * Como as categorias de uma variável se relacionam com as categorias de outra variável?
    * Observa-se o excesso ou falta de ocorrências de casos nas categorias das variáveis;
* Para cada valor de categoria $ij$, p valor de resíduo padronizado ajustado é dado pela formula:

$$
\text{Resíduo padronizado ajustado}_{ij}=\frac{\text{Resíduo padronizado}_{ij}}{\sqrt{[(1 - \frac{\sum_{i=0}^{i+1}}{N})(1 - \frac{\sum_{j=0}^{j+1}}{N})]}}
$$

* Se o valor do resíduo padronizado em uma célula for maior que $1,96$, existe associação significativa.
  * O valor crítico de uma distribuição normal padrão ($\mathcal{N}$) é $1,96$

In [ ]:
contingency_table.standardized_resids

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))

data = contingency_table.standardized_resids.values

sns.heatmap(
    contingency_table.standardized_resids,
    annot=True,
    mask=np.ma.masked_where(data < 1.96, data).mask,
    cmap='rocket',
    ax=ax
)

plt.show()

#### Elaboração e interpretação do mapa perceptual

##### Determinar os autovalores (_eigenvalues_) - $λ^2$

* Os autovalores referem-se à quantidade de inércias principais parciais
  * Base, também, para determinar:
    * Inércia principal total
    * Percentual da inércia principal total em cada dimensão do mapa perceptual
* A quantidade de autovalores depende da quantidade de categorias nas variáveis;
  * Pode-se obter através da formula:
    * $\text{Quantidade de autovalores} = m = min(i - 1, j - 1)$


* Para o cálculo dos autovalores, primeiro, determina-se uma matriz $A$:
  * Para cada valor dos resíduos padronizados

$$
\text{Matriz A}_{ij}=\frac{\text{Resíduo pradronizado}_{ij}}{\sqrt{N}}
$$

* Com base na matriz $A$, obtemos, também, a matriz $W$:

$$
\text{Matriz W}=A'A
$$

* Uma vez com a matriz $W$, os autovalores podem ser obtidos através da expressão:

$$
det(\lambda^2 . I - W) = 0
$$

$$
\begin{vmatrix}
\lambda^2-W_{12} & -W_{12} & -W_{13} \\
-W_{21} & \lambda^2-W_{22} & -W_{23} \\
-W_{31} & -W_{32} & \lambda^2-W_{33}\\
\end{vmatrix} = 0
$$

> $I$ é a matriz identidade

* Com base nos autovalores ($\lambda^2$), encontra-se o percentual da inércia principal total de cada dimensão

$$
\text{% da inércia principal total (para cada dimensão)}=\frac{\lambda^2_{\text{dimensão}}}{\lambda^2_{\text{total}}}
$$

In [ ]:
display(ca.eigenvalues_summary)
ca.eigenvalues_

* Quanto maior a inércia principal total (e o $\chi^2$), mais forte será a associação entre as variáveis em análise

$$
\text{Inércia principal total}=\frac{\chi^2}{N}
$$

In [ ]:
ca.total_inertia_

##### Determinar as massas das variáveis em linha e coluna

* As massas representam a influência que cada categoria exerce sobre as demais categorias de sua variável;
* Com base nos totais da tabela de contingência, para categoria $ij$ das variáveis, as massas podem ser obtidas através das seguintes expressões:

$$
\text{Massa linha}_{\text{Categoria i}}=\frac{\sum^{i=0}_{i+1}}{N}
$$

$$
\text{Massa coluna}_{\text{Categoria j}}=\frac{\sum^{j=0}_{j+1}}{N}
$$

In [ ]:
display('Massas da linha', ca.row_masses_)
display('Massas da coluna', ca.col_masses_)

##### Determinar os autovetores (_eigenvectors_)

* Os autovetores podem ser encontrados através dos autovalores ($\lambda^2$)
  * Cada autovalor gera seus próprios autovetores
* Para cada autovalor:
  * Substitui-se o autovalor na matriz $(\lambda^2.I-W)$
  * Multiplica-se por um vetor coluna e iguala a 0
  * Resolvendo o sistema de equações gerado, encontra-se o autovetor da coluna (V)
  * Com base neste autovetor, é possível encontrar o autovetor linha (U)

$$
u_k = \text{Matriz A} . (\frac{V_k}{\lambda_k})
$$

In [ ]:
display('Linha', ca.svd_.U)
display('Coluna', ca.svd_.V.T)

##### Obter as coordenadas das categorias - variáveis em linha

* Coordenadas das abscissas (X)

$$
X_i=(\lambda_1 . \frac{1}{\sqrt{D_i}}) . u_1
$$

* Coordenadas das ordenadas (Y)

$$
Y_i=(\lambda_2 . \frac{1}{\sqrt{D_i}}) . u_2
$$

* Coordenadas da $k$-ésima dimensão ($k$ sendo a quantidade de autovalores)

$$
Z_i=(\lambda_k . \frac{1}{\sqrt{D_i}}) . u_k
$$


In [ ]:
ca.row_coordinates(tabela)

##### Obter as coordenadas das categorias - variáveis em coluna

* Coordenadas das abscissas (X)

$$
X_j=(\lambda_1 . \frac{1}{\sqrt{D_j}}) . v_1
$$

* Coordenadas das ordenadas (Y)

$$
Y_j=(\lambda_2 . \frac{1}{\sqrt{D_j}}) . v_2
$$

* Coordenadas da $k$-ésima dimensão ($k$ sendo a quantidade de autovalores)

$$
Z_j=(\lambda_k . \frac{1}{\sqrt{D_j}}) . u_k
$$

In [ ]:
ca.column_coordinates(tabela)

##### Mapa perceptual

In [ ]:
def set_vars_labels(df, ax):
  label, x, y = df.columns
  for _idx, row in df.iterrows():
    ax.text(
      row[x] + .03,
      row[y] + .02,
      row[label],
      fontsize=12
    )

In [ ]:
row_coords = ca.row_coordinates(tabela)

chart_row = pd.DataFrame({
  'var_row': tabela.index,
  'x_row': row_coords[0].values,
  'y_row': row_coords[1].values
})

chart_row

In [ ]:
column_coords = ca.column_coordinates(tabela)

chart_column = pd.DataFrame({
  'var_column': tabela.columns,
  'x_column': column_coords[0].values,
  'y_column': column_coords[1].values
})

chart_column

In [ ]:
chart_obs = tabela.rename(
  columns={
    'Perfil': 'var_row',
    'Tipo de Aplicação': 'var_column'
  }
)

chart_obs = pd.merge(chart_obs, chart_row, how='left', on='var_row')
chart_obs = pd.merge(chart_obs, chart_column, how='left', on='var_column')

chart_obs['x_obs'] = chart_obs[['x_row', 'x_column']].mean(axis=1)
chart_obs['y_obs'] = chart_obs[['y_row', 'y_column']].mean(axis=1)

chart_obs.head()

In [ ]:
fig, ax = plt.subplots(figsize=(18, 10))

sns.scatterplot(
  chart_row,
  x='x_row',
  y='y_row',
  s=50,
  ax=ax
)
sns.scatterplot(
  chart_column,
  x='x_column',
  y='y_column',
  s=50,
  ax=ax
)

sns.scatterplot(
  chart_obs,
  x='x_obs',
  y='y_obs',
  s=50,
  ax=ax
)

set_vars_labels(chart_row, ax)
set_vars_labels(chart_column, ax)

sns.despine(top=True,
            right=True)

plt.axhline(y=0, color='k', ls='--')
plt.axvline(x=0, color='k', ls='--')

plt.title('Mapa perceptual', fontsize=18)
plt.xlabel(f'Dimensão 1 - {ca.eigenvalues_summary.iloc[0, 1]} da inércia', fontsize=12)
plt.ylabel(f'Dimensão 2 - {ca.eigenvalues_summary.iloc[1, 1]} da inércia', fontsize=12)

plt.show()

### Análise de Correspondência Múltipla (ACM)

#### Introdução

* A análise de correspondência múltipla (também referenciada como ACM), busca apontar a existência de associação entre mais de duas variáveis;
  * Apenas variáveis com significância estatística, com pelo menos outra variável, participam da análise;

#### Cálculo da siginifiância estatística $\chi^2$

##### Método da matriz binária Z - Coordenadas padrão

* Obtida através da transformação das variáveis qualitativas em binárias;
  * Também conhecidas como variáveis _dummies_ - 0 ou 1;
* Considerando que tal matriz será a tabela de contigência, será possível obter, posteriormente a transformação, a inércia principal parcial das dimensões, autovalores, autovetores e, por conseguinte, as coordenadas no mapa perceptual;
* A quantidade de dimensões ($\lambda^2$) é dada pela seguinte expressão:
  * $\lambda^2=J-Q$
  * $J$ sendo a quantidade total de categorias em todas as variáveis
  * $Q$ sendo a quantidade de variáveis

* A inércia principal total pode ser obtida através da seguinte expressão:

$$
\text{Inércia principal total}=\frac{J-Q}{Q}
$$

##### Método de [Burt](https://en.wikipedia.org/wiki/Multiple_correspondence_analysis) - Coordenadas principais

* Definida como sendo a matriz obtida através de:
  * $B=Z'.Z$
* Nesta matriz, obtem-se o cruzamento de todos os pares de variáveis e suas categorias;
  * Assim portanto, uma matriz que contém as frequências absolutas observadas para todos os cruzamentos
* Considerando que tal matriz será a tabela de contigência, será possível obter, posteriormente a transformação, a inércia principal parcial das dimensões, autovalores, autovetores e, por conseguinte, as coordenadas no mapa perceptual;